In [21]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [22]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Connected
Simulation time in milliseconds:  22700
Omnirob is at [x,y,z]= [-5.0000996589660645, -7.001837730407715, 0.35244911909103394]
Disconnected


In [23]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [10]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

# desired_position = r.get_object_position('Sphere1')
# print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

# relative_position = r.get_object_relative_position('Sphere1')
# print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

desired_position = (10.00, -7.00, 0)

# Virtual disk
disk = robot('Disc', client_id=r.client_id)


# Time duration
tf = 10
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    disk.set_position(point_traj)
    
    
    # Sensing
    robot_position = r.get_position()
    
         
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    
    
    time.sleep(tf/500)

r.send_motor_velocities([0, 0, 0, 0])

r.close_connection() 

Robot connected
Robot position: (-5.00, -7.00) 
Robot connected
[-5.00004482 -7.00068045  0.35244915]
[-4.96998461 -7.00067908  0.35174284]
[-4.9399244  -7.00067772  0.35103653]
[-4.90986419 -7.00067636  0.35033022]
[-4.87980398 -7.00067499  0.34962391]
[-4.84974377 -7.00067363  0.34891759]
[-4.81968356 -7.00067226  0.34821128]
[-4.78962335 -7.0006709   0.34750497]
[-4.75956314 -7.00066954  0.34679866]
[-4.72950293 -7.00066817  0.34609235]
[-4.69944272 -7.00066681  0.34538604]
[-4.66938251 -7.00066545  0.34467973]
[-4.6393223  -7.00066408  0.34397342]
[-4.60926209 -7.00066272  0.34326711]
[-4.57920188 -7.00066136  0.3425608 ]
[-4.54914167 -7.00065999  0.34185449]
[-4.51908146 -7.00065863  0.34114817]
[-4.48902125 -7.00065727  0.34044186]
[-4.45896104 -7.0006559   0.33973555]
[-4.42890083 -7.00065454  0.33902924]
[-4.39884062 -7.00065317  0.33832293]
[-4.36878041 -7.00065181  0.33761662]
[-4.3387202  -7.00065045  0.33691031]
[-4.30865999 -7.00064908  0.336204  ]
[-4.27859978 -7.00064772

[ 1.49296055 -7.0003859   0.19988599]
[ 1.52302076 -7.00038454  0.19917968]
[ 1.55308097 -7.00038318  0.19847337]
[ 1.58314118 -7.00038181  0.19776706]
[ 1.61320139 -7.00038045  0.19706075]
[ 1.6432616  -7.00037909  0.19635444]
[ 1.67332181 -7.00037772  0.19564812]
[ 1.70338202 -7.00037636  0.19494181]
[ 1.73344223 -7.000375    0.1942355 ]
[ 1.76350244 -7.00037363  0.19352919]
[ 1.79356265 -7.00037227  0.19282288]
[ 1.82362286 -7.0003709   0.19211657]
[ 1.85368307 -7.00036954  0.19141026]
[ 1.88374328 -7.00036818  0.19070395]
[ 1.91380349 -7.00036681  0.18999764]
[ 1.9438637  -7.00036545  0.18929133]
[ 1.97392391 -7.00036409  0.18858502]
[ 2.00398412 -7.00036272  0.1878787 ]
[ 2.03404433 -7.00036136  0.18717239]
[ 2.06410454 -7.00036     0.18646608]
[ 2.09416475 -7.00035863  0.18575977]
[ 2.12422496 -7.00035727  0.18505346]
[ 2.15428517 -7.0003559   0.18434715]
[ 2.18434538 -7.00035454  0.18364084]
[ 2.21440559 -7.00035318  0.18293453]
[ 2.2444658  -7.00035181  0.18222822]
[ 2.27452601

[ 7.98596593 -7.00009136  0.04732283]
[ 8.01602614 -7.00009     0.04661652]
[ 8.04608635 -7.00008864  0.04591021]
[ 8.07614656 -7.00008727  0.0452039 ]
[ 8.10620677 -7.00008591  0.04449759]
[ 8.13626698 -7.00008454  0.04379128]
[ 8.16632719 -7.00008318  0.04308497]
[ 8.1963874  -7.00008182  0.04237866]
[ 8.22644761 -7.00008045  0.04167234]
[ 8.25650782 -7.00007909  0.04096603]
[ 8.28656803 -7.00007773  0.04025972]
[ 8.31662824 -7.00007636  0.03955341]
[ 8.34668845 -7.000075    0.0388471 ]
[ 8.37674866 -7.00007364  0.03814079]
[ 8.40680887 -7.00007227  0.03743448]
[ 8.43686908 -7.00007091  0.03672817]
[ 8.46692929 -7.00006954  0.03602186]
[ 8.4969895  -7.00006818  0.03531555]
[ 8.52704971 -7.00006682  0.03460924]
[ 8.55710992 -7.00006545  0.03390292]
[ 8.58717013 -7.00006409  0.03319661]
[ 8.61723034 -7.00006273  0.0324903 ]
[ 8.64729055 -7.00006136  0.03178399]
[ 8.67735076 -7.00006     0.03107768]
[ 8.70741097 -7.00005864  0.03037137]
[ 8.73747118 -7.00005727  0.02966506]
[ 8.76753139

In [24]:
def move(desired_position):
    # Joint name in coppelia
    motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
    r = robot('Omnirob', motor_names)  # Create an instance of our robot

    # Initial locations
    robot_position = r.get_position()
#     print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

    # desired_position = r.get_object_position('Sphere1')
    # print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

    # relative_position = r.get_object_relative_position('Sphere1')
    # print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

#     desired_position = (10.00, -7.00, 0)

    # Virtual disk
    disk = robot('Disc', client_id=r.client_id)


    # Time duration
    tf = 5
    # Time interval
    time_steps = linspace(0, tf, 500)

    a1 = (desired_position - robot_position) / tf
    a0 = robot_position

    for t in time_steps:
        point_traj = a1 * t + a0
        vel_traj = a1
#         print(point_traj)

        disk.set_position(point_traj)


        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 10 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])



        time.sleep(tf/500)

    r.send_motor_velocities([0, 0, 0, 0])

    r.close_connection() 

In [26]:
move((10, -7, 0))
move((10, -3, 0))
move((-1, -3, 0))
move((-1, 7, 0))
move((3.5, 7, 0))
move((3.5, 0, 0))
move((-1, 0, 0))
move((-1, 10, 0))
move((9, 10, 0))

Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
Robot connected
Robot connected
Connection closed
